In [ ]:
import numpy as np
import shap
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from tensorflow import keras
from scipy.ndimage import gaussian_filter
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix

## Ładowanie datasetu

In [ ]:
data = tf.keras.datasets.mnist.load_data()

In [ ]:
train_data, test_data = data
X_train, y_train = train_data
X_test, y_test = test_data

X_train = X_train.reshape(-1, 28, 28, 1).astype(np.float32)
X_test = X_test.reshape(-1, 28, 28, 1).astype(np.float32)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

## Konfiguracja modelu sieci

In [ ]:
input_shape = X_train[0].shape
num_classes = 10

model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

## Trening modelu

Jeśli korzystacie z własnej architektury sieci przeprowadźcie trening od nowa

In [ ]:
history = model.fit(
    X_train,
    y_train,
    epochs=20,
    batch_size=256,
    validation_data=(X_test, y_test)
)

In [ ]:
x = np.arange(20)
plt.plot(x, history.history['loss'], color='red', label='train loss')
plt.plot(x, history.history['val_loss'], color='blue', label='validation loss')
plt.legend()
plt.show()

plt.plot(x, history.history['accuracy'], color='red', label='train accuracy')
plt.plot(x, history.history['val_accuracy'], color='blue', label='validation accuracy')
plt.legend()
plt.show()

## Ładowanie gotowych wag modelu

Dla leniwych - gotowe wagi

In [ ]:
model.load_weights('mnist_weights.h5')

Szybki test czy wszystko działa

In [ ]:
test_preds = model.predict(X_test)
test_preds_classes = np.argmax(test_preds, axis=-1)
test_accuracy = np.mean(test_preds_classes == y_test)

print('test accuracy:', round(test_accuracy, 2))

# Zadanie 1

Przeprowadź explaining za pomocą DeepExplainera dla przykładowych próbek. Uzupełnij fragmenty kodu zastąpione ???

In [ ]:
# Konstrukcja deep explainera
background = ??? # wybieramy 100 losowych próbek ze zbioru treningowego, z których będą samplowane wartości neutralne
explainer = shap.DeepExplainer(model, background)

In [ ]:
# Wybór losowych próbek ze zbioru testowego
SAMPLES_COUNT = 5
samples_batch = ???

In [ ]:
%%time
# Obliczenie shap values dla batcha wybranych próbek
samples_batch_shap = explainer.shap_values(???)

In [ ]:
labels = np.array([list(range(10))] * SAMPLES_COUNT)
shap.image_plot(samples_batch_shap, samples_batch / 255., labels=labels)

# Zadanie 2

Wyznacz przynajmniej dwie podobne klasy, które często są ze sobą mylone (wykorzystaj PCA, t-SNE, confusion matrix, lub inną metodę). Następnie znajdź kilka przykładowych próbek należących do z każdej ze znalezionych klas, dla których przy pomocy DeepShap możemy wyraźnie wyznaczyć obszary obrazów pozwalające jednoznacznie wskazać różnice pomiędzy cechami wybranych klas. W badaniu rezultatów może pomóc zaaplikowanie na shap values rozmycia gaussowskiego (warto przetestować wyniki dla różnych wartości parametru sigma).

In [ ]:
# Wyznaczenie podobnych klas

???

In [ ]:
similar_classes_indices = [???, ???]

In [ ]:
# Funkcje pomocnicze
def prep_img(img):
    return np.array([img / 255])


def compare_shap(shap_values, explained_classes, imgs):
    '''
    Rysuje wykres shap values wyjaśniających zadane klasy
    
    Parameters
    ----------
    shap_values : List[ndarray]
        Lista, której i-ty element to wyjaśnienia klasy i-tej dla kolejnych próbek 
        (wynik wywołania funkcji explainer.shap_values).
    explained_classes : List[int]
        Lista indeksów klas, które chcemy uwzględnić na wykresie wynikowym.
    imgs : List[ndarray]
        Lista obrazów reprezentujących wyjaśniane próbki.
    '''
    np_shap = np.array(shap_values)
    class_labels = explained_classes
    classes_shap = np_shap[explained_classes]
    for i, img in enumerate(imgs):
        img_shap = list(classes_shap[:, i:i+1])
        shap.image_plot(img_shap, prep_img(img), np.array([class_labels]))
        
        
def blur_shap_values(shap_values, sigma):
    '''
    Aplikuje rozmycie gaussowskie dla uzyskanych shap values
    
    Parameters
    ----------
    shap_values : List[ndarray]
        Lista, której i-ty element to wyjaśnienia klasy i-tej dla kolejnych próbek 
        (wynik wywołania funkcji explainer.shap_values).
    sigma : float
        Określa intensywność rozmycia.
    '''
    classes_count = len(shap_values)
    samples_count = shap_values[0].shape[0]
    blurred_shap_values = []
    for class_idx in range(classes_count):
        blurred = []
        for sample_idx in range(samples_count):
            blurred.append(gaussian_filter(shap_values[class_idx][sample_idx], sigma=sigma))
        blurred_shap_values.append(blurred)
    return blurred_shap_values

In [ ]:
SAMPLES_PER_CLASS = 5

# Dla każdej ze znalezionych klas wybranie i wyjaśnienie SAMPLES_PER_CLASS losowych próbek
for class_idx in similar_classes_indices:
    class_samples = X_test[y_test == class_idx]
    selected_class_samples = class_samples[???] # wybierz SAMPLES_PER_CLASS losowych próbek
    
    selected_samples_shap = explainer.shap_values(selected_class_samples)
    blurred_selected_samples_shap = blur_shap_values(selected_samples_shap, sigma=???) # dobierz parametr rozmycia
    compare_shap(blurred_selected_samples_shap, similar_classes_indices, selected_class_samples)

# Zadanie 3

Przetestuj działanie DeepExplainera dla próbek wyznaczonych zgodnie z następującymi zasadami (po kilka próbek):
1. Próbki dowolnych klas, które zostały poprawnie sklasyfikowane przez model z najwiekszą pewnością.
1. Próbki dowolnych klas, które zostały niepoprawnie sklasyfikowane przez model z najwiekszą pewnością przynależności do błędnej klasy.
1. Próbki dowolnych klas, dla których model waha się pomiędzy dwoma klasami (np. takie próbki, dla których confidence modelu wynosi przynajmniej 40% dla dwóch klas).

Zinterpretuj otrzymane wyniki, czy takie próbki dobrze nadają się do analizy za pomocą shapa? Czy można stwierdzić dlaczego zostały dobrze/źle zaklasyfikowane?

#### 1. Próbki dowolnych klas, które zostały poprawnie sklasyfikowane przez model z najwiekszą pewnością.

In [ ]:
???

#### 2. Próbki dowolnych klas, które zostały niepoprawnie sklasyfikowane przez model z najwiekszą pewnością przynależności do błędnej klasy.

In [ ]:
???

#### 3. Próbki dowolnych klas, dla których model waha się pomiędzy dwoma klasami (np. takie próbki, dla których confidence modelu wynosi przynajmniej 40% dla dwóch klas).

In [ ]:
???

# Zadanie 4 [Dla chętnych]

1. Przetestuj działanie DeepExplainera dla zastosowania różnych backgroundów (cały czarny obraz, średnia całego zbioru danych, mediana itp.). 
2. Sprawdź, czy użycie KernelExplainera dla przykładowych obrazów pozwala uzyskać sensowne wyniki (będzie to wymagało spłaszczenia danych).

In [ ]:
???